In [ ]:
pip install python-dotenv

In [ ]:
print("ho")

ho


In [ ]:
import requests
import pprint
from difflib import SequenceMatcher
import pandas as pd
import time

URL = 'https://api.semanticscholar.org/graph/v1/paper/'
HEADERS = { 'x-api-key':  'XpAdCTxvUU2udInEBaKwa97vtVe4MvRO8rDESYWK' }
CLUSTER_FIELDS = ','.join(["title", "abstract"])
# DB_FIELDS = ','.join(["title", "year", "authors", "citations.paperId", "citations.title", "references.paperId", "references.title"])

def str_sim(strA: str, strB: str) -> float:
    return SequenceMatcher(None, strA, strB).ratio()

def query_papers(query: str):
    PARAMS = {
        "query": query,
        "limit": 5,
        "fields": CLUSTER_FIELDS
    }
    response = requests.get(URL + "search", headers=HEADERS, params=PARAMS)

    if response.ok:
        data = response.json()
        if "data" in data:
            return data["data"]
        else:
            print("No papers found for query:", query)
            return []  # Return an empty list if no data is found
    else:
        raise Exception("Error: " + response.text)


# # Documentation: https://api.semanticscholar.org/api-docs/#tag/Paper-Data/operation/get_graph_get_paper
# def get_paper(paperID: str):
#     PARAMS = {
#         "CLUSTER_FIELDS": CLUSTER_FIELDS
#     }
#     response = requests.get(URL + paperID, headers=HEADERS, params=PARAMS)

#     if response.ok:
#         return response.json()
#     else:
#         raise Exception("Error: " + response.text)

def get_matching_paper(title: str):
    data = query_papers(title)
    target = max(data, key=lambda entry: str_sim(title, entry['title']))
    return target

def produce(csv_name):
    df = pd.read_csv(csv_name)
    titles = df['Title'].tolist()
    i = 1
    for title in titles:
        data = get_matching_paper(title)
        yield data['paperId'], data['title'], data['abstract']
        if i % 50 == 0:
            print('Finished', i)
        time.sleep(0.5)
        i += 1

def query_papers(query: str):
    PARAMS = {
        "query": query,
        "limit": 5,
        "fields": CLUSTER_FIELDS
    }
    response = requests.get(URL + "search", headers=HEADERS, params=PARAMS)

    if response.ok:
        data = response.json()
        print(data)  # Print the response to inspect its structure
        return data["data"]
    else:
        raise Exception("Error: " + response.text)




if __name__ == "__main__":
    df = pd.DataFrame(produce("mlsys_papers.csv"), columns=["ID", "Title", "Abstract"])
    df.to_csv("mlsys_cluster.csv", index=False)

{'total': 0, 'offset': 0}


KeyError: 'data'

In [ ]:
import os
import re

import requests

S2_API_KEY = os.getenv('S2_API_KEY')
result_limit = 10


def main():
    basis_paper = find_basis_paper()
    find_recommendations(basis_paper)


def find_basis_paper():
    papers = None
    while not papers:
        query = input('Find papers about what: ')
        if not query:
            continue

        rsp = requests.get('https://api.semanticscholar.org/graph/v1/paper/search',
                           headers={'X-API-KEY': S2_API_KEY},
                           params={'query': query, 'limit': result_limit, 'fields': 'title,url'})
        rsp.raise_for_status()
        results = rsp.json()
        total = results["total"]
        if not total:
            print('No matches found. Please try another query.')
            continue

        print(f'Found {total} results. Showing up to {result_limit}.')
        papers = results['data']
        print_papers(papers)

    selection = ''
    while not re.fullmatch('\\d+', selection):
        selection = input('Select a paper # to base recommendations on: ')
    return results['data'][int(selection)]


def find_recommendations(paper):
    print(f"Up to {result_limit} recommendations based on: {paper['title']}")
    rsp = requests.get(f"https://api.semanticscholar.org/recommendations/v1/papers/forpaper/{paper['paperId']}",
                       headers={'X-API-KEY': S2_API_KEY},
                       params={'fields': 'title,url', 'limit': 10})
    rsp.raise_for_status()
    results = rsp.json()
    print_papers(results['recommendedPapers'])


def print_papers(papers):
    for idx, paper in enumerate(papers):
        print(f"{idx}  {paper['title']} {paper['url']}")


if __name__ == '__main__':
    main()

Find papers about what: robots
Found 755036 results. Showing up to 10.
0  Design, fabrication and control of soft robots https://www.semanticscholar.org/paper/9ba6d7604f06479e64e292dd78d8a48c612dd374
1  Real-Time Obstacle Avoidance for Manipulators and Mobile Robots https://www.semanticscholar.org/paper/c8a04d0cbb9f70e86800b11b594c9a05d7b6bac0
2  Magnetic Soft Materials and Robots. https://www.semanticscholar.org/paper/7ef04bce51f857f6b156c158aa86499fa3c8dee5
3  IEEE/RSJ International Conference on Intelligent Robots and Systems https://www.semanticscholar.org/paper/dda867de88aff74f70fb9b2de0387297183afea5
4  Robots and Jobs: Evidence from US Labor Markets https://www.semanticscholar.org/paper/be9aa6c68d4df56a55444b15a83ff1e62c869bb5
5  RMA: Rapid Motor Adaptation for Legged Robots https://www.semanticscholar.org/paper/1ca5ff6555d9fc634d3858d1fda9b3de2a91b13a
6  Understanding anthropomorphism in service provision: a meta-analysis of physical robots, chatbots, and other AI https://www.s

In [ ]:
import requests
import pandas as pd
import time
from difflib import SequenceMatcher

URL = 'https://api.semanticscholar.org/graph/v1/paper/'
HEADERS = {'x-api-key': '5RTw6MHqO44aHnMCRyXsC4x3nsFp1ZP7a0eOK6ol'}
CLUSTER_FIELDS = ','.join(["title", "abstract"])


def str_sim(strA: str, strB: str) -> float:
    return SequenceMatcher(None, strA, strB).ratio()


def query_papers(query: str):
    PARAMS = {
        "query": query,
        "limit": 1,  # Limit to 1 result as we are looking for a specific paper
        "fields": CLUSTER_FIELDS
    }
    response = requests.get(URL + "search", headers=HEADERS, params=PARAMS)

    if response.ok:
        data = response.json()
        return data.get("data", [])  # Use .get() to avoid KeyError if data is empty
    else:
        raise Exception("Error: " + response.text)


def get_matching_paper(title: str):
    data = query_papers(title)
    if data:
        return data[0]  # Return the first (and only) result if found
    else:
        print("No matching paper found for title:", title)
        return None


def get_paper_info(title: str):
    paper = get_matching_paper(title)
    if paper:
        return paper['paperId'], paper['title'], paper['abstract']
    else:
        return None, None, None


def produce(csv_name):
    df = pd.read_csv(csv_name)
    for title in df['Title']:
        paper_id, title, abstract = get_paper_info(title)
        yield paper_id, title, abstract
        time.sleep(0.5)  # Add a delay to avoid overwhelming the API


if __name__ == "__main__":
    df = pd.DataFrame(produce("mlsys_papers.csv"), columns=["ID", "Title", "Abstract"])
    df.to_csv("mlsys_cluster.csv", index=False)


No matching paper found for title: Accelerating deep learning inference via freezing." 11th {USENIX} Workshop on Hot Topics in Cloud Computing
No matching paper found for title: DeepCache: Principled cache for mobile deep vision." Proceedings of the 24th Annual International Conference on Mobile Computing and Networking
No matching paper found for title: Swearingen, Thomas, et al. "ATM: A distributed, collaborative, scalable system for automated machine learning.
No matching paper found for title: Lianmin Zheng, UC Berkeley; Chengfan Jia, Minmin Sun, and Zhao Wu, Alibaba Inc.; Cody Hao Yu, Amazon Web Services, Inc; Ameer Haj-Ali, UC Berkeley; Yida Wang, Amazon Web Services, Inc; Jun Yang, Alibaba Inc.; Danyang Zhuo, Duke University and UC Berkeley; Koushik Sen, Joseph Gonzalez, and Ion Stoica, UC Berkeley
No matching paper found for title: HiveD: Sharing a GPU Cluster for Deep Learning with Guarantees


In [ ]:
#all values
import requests
import pandas as pd
import time
from difflib import SequenceMatcher
#XpAdCTxvUU2udInEBaKwa97vtVe4MvRO8rDESYWK
#5RTw6MHqO44aHnMCRyXsC4x3nsFp1ZP7a0eOK6ol
URL = 'https://api.semanticscholar.org/graph/v1/paper/'
HEADERS = {'x-api-key': 'XpAdCTxvUU2udInEBaKwa97vtVe4MvRO8rDESYWK'}
CLUSTER_FIELDS = ','.join(["title", "abstract", "url", "venue", "year", "authors", "citations", "references"])

def str_sim(strA: str, strB: str) -> float:
    return SequenceMatcher(None, strA, strB).ratio()

def query_papers(query: str):
    PARAMS = {
        "query": query,
        "limit": 1,  # Limit to 1 result as we are looking for a specific paper
        "fields": CLUSTER_FIELDS
    }
    response = requests.get(URL + "search", headers=HEADERS, params=PARAMS)

    if response.ok:
        data = response.json()
        return data.get("data", [])  # Use .get() to avoid KeyError if data is empty
    else:
        raise Exception("Error: " + response.text)

def get_matching_paper(title: str):
    data = query_papers(title)
    if data:
        return data[0]  # Return the first (and only) result if found
    else:
        print("No matching paper found for title:", title)
        return None

def get_paper_info(title: str):
    paper = get_matching_paper(title)
    if paper:
        return {
            'Paper ID': paper['paperId'],
            'URL': paper['url'],
            'Title': paper['title'],
            'Venue': paper['venue'],
            'Year': paper['year'],
            'Authors': [{'Name': author['name'], 'Author ID': author['authorId']} for author in paper['authors']],
            'Citations': [citation['paperId'] for citation in paper['citations']],
            'References': [reference['paperId'] for reference in paper['references']],
            'Abstract': paper['abstract']
        }
    else:
        return None

def produce(csv_name):
    df = pd.read_csv(csv_name)
    for title in df['Title']:
        paper_info = get_paper_info(title)
        if paper_info:
            yield paper_info
        else:
            time.sleep(0.6)  # Add a delay to avoid overwhelming the API

if __name__ == "__main__":
    df = pd.DataFrame(produce("mlsys_papers.csv"))
    df.to_csv("mlsys_cluster.csv", index=False)


No matching paper found for title: Lianmin Zheng, UC Berkeley; Chengfan Jia, Minmin Sun, and Zhao Wu, Alibaba Inc.; Cody Hao Yu, Amazon Web Services, Inc; Ameer Haj-Ali, UC Berkeley; Yida Wang, Amazon Web Services, Inc; Jun Yang, Alibaba Inc.; Danyang Zhuo, Duke University and UC Berkeley; Koushik Sen, Joseph Gonzalez, and Ion Stoica, UC Berkeley
No matching paper found for title: HiveD: Sharing a GPU Cluster for Deep Learning with Guarantees
